In [1]:
# Imports and getting data
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv(os.path.expanduser("/Users/Home/HackOHIO/Prescriber_Data.csv"))

In [2]:
# Get labels, and associated data
df.columns = ['id', 'first_name', 'last_name', 'State', 'Product', 
              'nrx1', 'nrx2', 'nrx3', 'nrx4', 'nrx5', 'nrx6',
              'trx1', 'trx2', 'trx3','trx4', 'trx5', 'trx6']
nrx_labels = df.columns[5:11]
trx_labels = df.columns[11: 17]
nrx = df.iloc[:, 5:11]
trx = df.iloc[:, 11:17]

In [3]:
# Clean the data by removing false values
df_remove = df.loc[(df['nrx1'] <= df['trx1']) & 
                   (df['nrx2'] <= df['trx2']) &
                   (df['nrx3'] <= df['trx3']) &
                   (df['nrx4'] <= df['trx4']) &
                   (df['nrx5'] <= df['trx5']) &
                   (df['nrx6'] <= df['trx6'])]

In [4]:
# Make a copy of the dataframe
df_add = df.copy(deep = True)

# Clean the data, make sure that if nrx > trx, then add nrx to trx value
for i in range(len(df)):
    nrx_row = nrx.iloc[i, :]
    trx_row = trx.iloc[i, :]
    for j in range(len(nrx_row)):
        nrx_month = nrx_row.iloc[j]
        trx_month = trx_row.iloc[j]
        if (nrx_month > trx_month):
            # Add nrx to trx
            new_trx_month = nrx_month + trx_month
            df_add.iat[i, j + 11] = new_trx_month

In [5]:
# Make a copy of the dataframe
df_avg = df.copy(deep = True)

# Clean the data, make sure that if nrx > trx, then add nrx to trx value
for i in range(3):
    nrx_row = nrx.iloc[i, :]
    trx_row = trx.iloc[i, :]
    for j in range(len(nrx_row)):
        nrx_month = nrx_row.iloc[j]
        trx_month = trx_row.iloc[j]
        if (nrx_month > trx_month):
            # Add nrx to trx
            new_trx_month = np.mean(trx_row)
            df_avg.iat[i, j + 11] = new_trx_month

In [6]:
# Get slope and intercept for trx
x = [1,2,3,4,5,6]

dfs = [df_remove, df_add, df_avg]
for dataframe in dfs:
    nrx_slope = []
    nrx_intercept = []
    nrx_means = []
    trx_slope = []
    trx_intercept = []
    trx_means = []
    nrx = dataframe.iloc[:, 5:11]
    trx = dataframe.iloc[:, 11:17]
    for i in range(len(dataframe)):
        nrx_y = np.array(nrx.iloc[i])
        nrx_m, nrx_b = np.polyfit(x, nrx_y, 1)
        nrx_mean = np.mean(nrx.iloc[i])
    
        trx_y = np.array(trx.iloc[i])
        trx_m, trx_b = np.polyfit(x, trx_y, 1)
        trx_mean = np.mean(trx.iloc[i])
    
        nrx_slope.append(nrx_m)
        nrx_intercept.append(nrx_b)
        nrx_means.append(nrx_mean)
    
        trx_slope.append(trx_m)
        trx_intercept.append(trx_b)
        trx_means.append(trx_mean)
    dataframe['nrx_slope'] = nrx_slope
    dataframe['nrx_intercept'] = nrx_intercept
    dataframe['trx_slope'] = trx_slope
    dataframe['trx_intercept'] = trx_intercept

<ipython-input-6-347eb2899687>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['nrx_slope'] = nrx_slope
<ipython-input-6-347eb2899687>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['nrx_intercept'] = nrx_intercept
<ipython-input-6-347eb2899687>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [14]:
df_remove.to_csv('df_rm.csv')
df_add.to_csv('df_add.csv')
df_avg.to_csv('df_avg.csv')